"""

@author matheus dias
## Recomendador de Animes
---

<img src="https://i.gifer.com/4ezV.gif" width="750">



**---Português---**

Script com um sistema de recomendação simples baseado nos gêneros do anime e nas avaliações dos usuários

Os sistemas de recomendação estão nas mais avançadas empresas 
de e-commerce, tanto em produtos como em mercadorias. 
Entender o comportamento dos consumidores é essencial para que possamos modelar 
um sistema que acompanhe esse comportamento com a finalidade de 
satisfazer o consumidor e gerar lucros para a empresa.

---

**---English---**

Script with a simple recommendation system based on anime genres and user ratings

Recommender systems are in the most advanced e-commerce companies, 
both in products and merchandise. Understanding consumer behavior is essential 
so that we can model a system that monitors this behavior in order to satisfy 
the consumer and generate profits for the company.

"""


## Manipulação dos dados
----

In [ ]:
# importações necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# leitura dos CSV
dataframe_anime = pd.read_csv('anime.csv')
dataframe_notas = pd.read_csv('rating.csv')

In [ ]:
# visualizando dataframe
display(dataframe_anime.head())
dataframe_notas.head()

In [ ]:
# informações dos dados do dataframe
display(dataframe_anime.info())
print('=='*40)
display(dataframe_notas.info())

In [ ]:
# juntando os dataframes
dataframe = pd.merge(dataframe_notas, dataframe_anime.drop('rating', axis=1),on='anime_id' )
print(dataframe.shape)
dataframe.head()

In [ ]:

# testando a junção
dataframe.groupby('name')['rating'].count().sort_values(ascending=False).head(600)

In [ ]:
# dataframe de animes e suas avaliações
notas = pd.DataFrame(dataframe.groupby('name')['rating'].mean())
notas['Número de Avaliações'] = pd.DataFrame(dataframe.groupby('name')['rating'].count())
notas['Média de Avaliação'] = pd.DataFrame(dataframe.groupby('name')['rating'].mean().round(2))
notas

In [ ]:
# plotar dados de notas
fig = plt.figure(figsize=(10,5))
sns.distplot(notas['rating'])
plt.xlim(0,10)
plt.title('Distribuição das notas (entre 0 e 10)', fontsize=15);

In [ ]:
# dataframe de animes e seus gêneros
genero = pd.DataFrame(data=dataframe_anime[['name','genre']])
genero.set_index('name', inplace=True)
genero

-----------------------------------------------------------------------
## Sistema de recomendação
-----------------------------------------------------------------------

In [ ]:
def verificar_genero(lista_genero, string):
    """
    Verifica se algum dos gêneros da lista está contido na string.

    Args:
        lista_genero (list): Uma lista de gêneros a serem verificados na string.
        string (str): A string na qual deseja-se verificar a presença dos gêneros.

    Returns:
        bool: True se pelo menos um dos gêneros da lista estiver contido na string, False caso contrário.

    Note:
        Esta função retorna True se pelo menos um dos gêneros da lista_genero estiver contido na string. Caso contrário,
        ela retorna False.
    """
    if any(x in string for x in lista_genero): 
        return True 
    else: 
        return False 

In [ ]:

def recomendar_anime(nome_do_anime, n):
    """
    Recomenda animes com base na correlação de avaliações dos usuários.

    Args:
        nome_do_anime (str): O nome do anime para o qual deseja receber recomendações.
        n (int): O número de animes recomendados desejados.

    Returns:
        pandas.DataFrame: Um DataFrame contendo os animes recomendados com suas informações de correlação,
        número de avaliações e média de avaliação. Os resultados são classificados por correlação em ordem decrescente.
        A primeira linha do DataFrame corresponde ao próprio anime consultado.

    Raises:
        KeyError: Se o nome_do_anime não existir no conjunto de dados ou se não houver informações suficientes para
        recomendações (menos de 3000 avaliações).

    Note:
        Esta função assume a existência de variáveis globais, como 'genero', 'dataframe_anime', 'dataframe' e 'notas',
        que devem ser definidas previamente.

    """
    genero_anime = genero.loc[nome_do_anime].values[0].split(', ')
    cols = dataframe_anime[dataframe_anime['genre'].apply(lambda x: verificar_genero(genero_anime, str(x)))]['name'].tolist()
    matriz_de_animes = dataframe[dataframe['name'].isin(cols)].pivot_table(index='user_id', columns='name', values='rating')
    anime_nota = matriz_de_animes[nome_do_anime]
    anime_parecido = matriz_de_animes.corrwith(anime_nota)
    anime_correlacionado = pd.DataFrame(anime_parecido, columns=['correlação'])
    anime_correlacionado = anime_correlacionado.join(notas[['Número de Avaliações', 'Média de Avaliação']])
    anime_correlacionado.dropna(inplace=True)
    animes_recomendados = anime_correlacionado[anime_correlacionado['Número de Avaliações'] > 3000].sort_values('correlação', ascending=False)
    animes_recomendados= animes_recomendados.rename_axis('Animes recomendados')
    print(f'Anime escolhido: {nome_do_anime}')
    return animes_recomendados.head(n+1)

-----------------------------------------------------------------------
## Testes
-----------------------------------------------------------------------

In [ ]:
# teste do código
dataframe.groupby('name')['rating'].count().sort_values(ascending=False).head()

In [ ]:
# teste do código
recomendar_anime('Steins;Gate', 5)

In [ ]:
# teste do código
recomendar_anime('Fullmetal Alchemist: Brotherhood', 3)